# Linear Regression

Author & Instructor: Diana NURBAKOVA, PhD.

In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="../styles/styles.css">

In [ ]:
import sys
from pathlib import Path

# Add the "resources" directory to the path
project_root = Path().resolve().parent
resources_path = project_root / 'resources'
sys.path.insert(0, str(resources_path))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, HTML

In [ ]:
from InteractiveRegression import (
    create_interactive_regression,
    create_static_visualization,
    compare_lines,
    plot_error_example,
    plot_3d_regression, 
    calculate_confidence_intervals, 
    plot_regression_with_intervals
)

In [ ]:
from polynomial import (generate_polynomials_plots, get_scenarios, get_poly_fit, get_overfit_example)

## Learning Objectives

By the end of this session, you will be able to:
- Construct a linear regression model using OLS
- Use polynomial features 
- Apply regularisation mechanisms

<div class="alert alert-info">
<h4>🎯 Will you love this film?</h3>

Netflix has millions of user ratings, but how do they predict if you'll rate a movie 4.2 stars vs 4.3 stars?

By the end, you'll understand how to build the core engine that powers recommendation systems.

</div>

Before addressing this problem, let's start with the following toy example. 

We have 9 observations of infants' height and age:

| Age | Height |
| ---- | ----- |
| 1.0 | 70.56 |
| 1.5 | 67.68 |
| 2.0 | 80.88 | 
| 2.5 | 82.32 |
| 3.0 | 84.00 |
| 3.5 | 90.00 | 
| 4.0 | 93.60 |
| 4.5 | 105.36 |
| 5.0 | 109.92 |


In [ ]:
# toy example data points
data_x = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]) # age
data_y = np.array([70.56, 67.68, 80.88, 82.32, 84.00, 90.00, 93.60, 105.36, 109.92]) # height


Let's plot height vs age.

In [ ]:
# visualisation of data points
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.scatter(data_x, data_y, c='blue', s=80, alpha=0.8, edgecolors='white', linewidth=2)
ax.set_xlim(0.5, 5.5)
ax.set_ylim(60, 115)
ax.set_xlabel("Age (years)")
ax.set_ylabel("Height (cm)")
ax.set_title("Age vs Height")
ax.grid(True, alpha=0.3)

It seems that there is a linear relation between these two measurements. 

We would like to create a linear model that represents our data or in other words, that allows us to see what a trend is. To do so, we can draw a line that **fits** the data.

Let's draw a couple of lines trying to represent (fit) the data points.


In [ ]:
# interactive visualisation of fitted line
create_interactive_regression(data_x, data_y)

In [ ]:
# compare several lines
compare_lines(data_x, data_y, None)

What makes one line mathematically 'better' than another?

## How to Find Your Line: Ordinary Least Squares (OLS)
<a id="ols"></a>

### Slope-Intercept Form of a Line
<a id="slope-intercept"></a>

Equation of a line: $$y = ax + b$$ 
where $y$ is a dependent variable, $x$ is an independent variable, $a$ is a slope and $b$ is an intercept.

<center>
<img src="img/line-equation.png" alt="Line equation", width="400">
</center>

**Geometric interpretation of slope and intercept**

<center>
<img src="img/slope-intercept.png" alt="Geometric interpretation of slope and intercept", width="400">
</center>

$$b = \text{where the line crosses the } y\text{-axis}$$
$$a = \frac{\text{Rise}}{\text{Run}}= \frac{\text{go up}}{\text{move to the side}} = \frac{\text{\# units over }y}{\text{\# units over }x}$$

### Finding Parameters
<a id="params-2d-case"></a>

> How to find parameters of a line?

In [ ]:
plot_error_example(data_x, data_y)

On the one hand, our estimated line is given by: 
$$\hat{y} = ax + b$$

On the other hand, there are observed data points $(x_i, y_i), i=\bar{1,n}$.

So, we want to minimize the difference between estimated and observed values, i.e. *the error*: $$\sum_{i=1}^{n}{err}_i^2 = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \sum_{i=1}^{n}(y_i - (ax_i + b))^2$$

Which means that we want to find the values of $a$ and $b$ that minimize this objective function. A candidate is given by:

$$\left\{\begin{array}{l} a = \frac{\sum_{i=1}^n(x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n}(x_i - \bar{x})^2} = \frac{Cov(X, Y)}{s_X^2} \\ b = \frac{\sum_{i=1}^n y_i}{n} - a\frac{\sum_{i=1}^n x_i}{n} = \bar{y} - a \bar{x}\end{array}\right.$$

Let's prove that these values of $a$ and $b$ minimize our goal function $f = \sum_{i=1}^n {err}_i^2 = \sum_{i=1}^{n}(y_i - (ax_i + b))^2$.

$$(\hat{a}, \hat{b}) = \argmin{\sum_{i=1}^n {err}_i^2} = \argmin{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2} = \argmin{\sum_{i=1}^{n}(y_i - (ax_i + b))^2}$$


<details>
<summary> Click to reveal the solution </summary>

1. Calculate the partial derivatives and set them equal to 0
$$\left\{\begin{array}{l}\frac{\partial f}{\partial a} = -2 \sum_{i=1}^{n}(y_i - ax_i - b)x_i = 0 \\ \frac{\partial f}{\partial b} =  -2 \sum_{i=1}^{n}(y_i - ax_i - b) = 0\end{array}\right.$$

First, let's focus on the second equation:
$$\sum_{i=1}^{n}(y_i - ax_i - b) = 0$$
which is equivalent to:
$$\sum_{i=1}^{n}y_i -\sum_{i=1}^n ax_i - nb = 0$$

Let's solve it for $b$:
$$b = \frac{\sum_{i=1}^{n}y_i -\sum_{i=1}^n ax_i}{n} = \frac{\sum_{i=1}^{n}y_i - a\sum_{i=1}^n x_i}{n} = \frac{\sum_{i=1}^{n}y_i}{n}- a\frac{\sum_{i=1}^n x_i}{n} = \bar{y} - a \bar{x}$$

Let's replace $b$ with its value in the first equation:
$$-2 \sum_{i=1}^{n}(y_i - ax_i - b)x_i = -2 \sum_{i=1}^{n}(y_i - ax_i - (\bar{y} - a \bar{x}))x_i = 0$$
$$\sum_{i=1}^{n}(y_i - ax_i - (\bar{y} - a \bar{x}))x_i = 0$$
$$\sum_{i=1}^{n}(y_ix_i - ax_ix_i - \bar{y}x_i + a \bar{x}x_i) = 0$$
$$\sum_{i=1}^{n}((y_ix_i - \bar{y}x_i) - (ax_ix_i - a \bar{x}x_i)) = 0$$
$$\sum_{i=1}^{n}((y_i - \bar{y})x_i - (ax_i - a \bar{x})x_i) = 0$$
$$\sum_{i=1}^{n}(y_i - \bar{y})x_i - \sum_{i=1}^{n}(ax_i - a \bar{x})x_i = 0$$
$$\sum_{i=1}^{n}(y_i - \bar{y})x_i - a\sum_{i=1}^{n}(x_i - \bar{x})x_i = 0$$

Let's solve it for $a$:
$$a = \frac{\sum_{i=1}^{n}(y_i - \bar{y})x_i}{\sum_{i=1}^{n}(x_i - \bar{x})x_i}$$

$$a = \frac{\sum_{i=1}^{n}(y_ix_i - \bar{y}x_i)}{\sum_{i=1}^{n}(x_i^2 - \bar{x}x_i)} = \frac{\sum_{i=1}^{n}y_ix_i - \bar{y}\sum_{i=1}^{n}x_i}{\sum_{i=1}^{n}x_i^2 - \bar{x}\sum_{i=1}^{n}x_i} = \frac{\sum_{i=1}^{n}y_ix_i - \bar{y}(n\bar{x})}{\sum_{i=1}^{n}x_i^2 - \bar{x}(n\bar{x})} = \frac{\sum_{i=1}^{n}y_ix_i - n\bar{y}\bar{x}}{\sum_{i=1}^{n}x_i^2 - n\bar{x}^2}$$

Given that: $$Cov(X, Y) = \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{n} = \frac{\sum_{i=1}^n(x_iy_i - x_i\bar{y} - \bar{x}y_i + \bar{x}\bar{y})}{n} = \frac{\sum_{i=1}^nx_iy_i - \sum_{i=1}^nx_i\bar{y} - \sum_{i=1}^n\bar{x}y_i + n\bar{x}\bar{y}}{n}=$$
$$=\frac{\sum_{i=1}^nx_iy_i - n(\frac{1}{n}\sum_{i=1}^nx_i)\bar{y} - n(\frac{1}{n}\sum_{i=1}^ny_i)\bar{x} + n\bar{x}\bar{y}}{n}=\frac{\sum_{i=1}^nx_iy_i - n\bar{x}\bar{y} - n\bar{y}\bar{x} + n\bar{x}\bar{y}}{n}=\frac{\sum_{i=1}^nx_iy_i - n\bar{x}\bar{y}}{n}$$

$$a = \frac{nCov(X, Y)}{ns^2_X} = \frac{Cov(X, Y)}{s^2_X}$$

Thus, we obtain a critical point $(\hat{a}, \hat{b}) = \big(\frac{Cov(X, Y)}{s^2_X}, \bar{y} - \bar{x}\frac{Cov(X, Y)}{s^2_X}\big)$


2. Apply the second derivative test:
- For each critical point, calculate the second partial derivatives:

$$f_{aa} = \frac{\partial^2f}{\partial a^2}$$
$$f_{bb} = \frac{\partial^2f}{\partial b^2}$$
$$f_{ab} = \frac{\partial^2f}{\partial a\partial b}$$

- Then construct the Hessian matrix $\mathcal{H}$ and compute its determinant (discriminant): $D = f_{aa}\dot f_{bb} - f_{ab}^2$
- At each critical point:

    * If $D > 0$ and $f_{aa} > 0$ or equivalently, $tr(\mathcal{H}) = f_{aa} + f_{bb} > 0$: local minimum
    * If $D > 0$ and $f_{aa} < 0$ or equivalently, $tr(\mathcal{H}) = f_{aa} + f_{bb} < 0$: local maximum
    * If $D < 0$: saddle point
    * If $D = 0$: test is inconclusive

Recall that $f = \sum_{i=1}^{n}(y_i - (ax_i + b))^2$. Then:

$$f_{aa} = \frac{\partial^2f}{\partial a^2} = (-2 \sum_{i=1}^{n}(y_i - ax_i - b)x_i)'_a = 2\sum_{i=1}^{n}x_i^2 \mathbb{> 0}$$

$$f_{bb} = \frac{\partial^2f}{\partial b^2} = (-2 \sum_{i=1}^{n}(y_i - ax_i - b))'_b = 2n$$

$$f_{ab} = \frac{\partial^2f}{\partial a\partial b} = (-2 \sum_{i=1}^{n}(y_i - ax_i - b)x_i)'_b = 2\sum_{i=1}^2 x_i = 2n\bar{x}$$

So, we obtain the following Hessian matrix: $$\mathcal{H}(a, b) = \left(\begin{matrix}\frac{\partial^2f}{\partial a^2} & \frac{\partial^2f}{\partial a\partial b} \\ \frac{\partial^2f}{\partial b\partial a} & \frac{\partial^2f}{\partial b^2}\end{matrix}\right) = \left(\begin{matrix}2\sum_{i=1}^{n}x_i^2 & 2n\bar{x} \\ 2n\bar{x} & 2n\end{matrix}\right)$$

Now, let's compute its determinant:

$$D = det(\mathcal{H}) = f_{aa}\dot f_{bb} - f_{ab}^2 = 4n\sum_{i=1}^{n}x_i^2 - 4 n^2 \bar{x}^2 = 4n^2 (\frac{1}{n}\sum_{i=1}^{n}x_i^2 - \bar{x}^2) = 4n^2Var(X) \mathbb{> 0}$$

Thus, we obtain $D > 0, f_{aa} > 0$ and $tr(\mathcal{H}) = 2\sum_{i=1}^{n}x_i^2 + 2n > 0$. Therefore, we can conclude that our critical point $(\hat{a}, \hat{b})$ is a local minimum of $f$. 

3. Check boundary conditions, if the domain has boundaries.

Since this function is defined on all of $\mathbb{R}^2$ with no constraints, we only need the critical point analysis.

4. Compare and identify global minimum / maximum by evaluating $f(x,y)$ at all local minima / maxima found in the previous steps.

In our case, there is only one critical point. So, our point is also a global minimum. 

</details>

Now, let's check the optimal solution of a line for our toy example:

In [ ]:
# Create static visualization first
print("Optimal solution:")
create_static_visualization(data_x, data_y)

<div class="alert-exercise">
<h5> QUESTION 1:</h5> Calculate manually the coefficients for our toy example and compare the results of each step with python calculations.
</div>


In [ ]:
# ANSWER


## From 1 Independent Variable to $n$ (General Case)
<a id="ols-general"></a>

Now, let's add weight to our toy example and see what our linear model is going to look like.

In [ ]:
# weight
data_z = np.array([12.0, 13.5, 15.0, 16.5, 18.0, 19.8, 21.0, 22.5, 24.0])

In [ ]:
# plot regression plane
plot_3d_regression(data_x, data_y, data_z)

Previously, we have seen that in case of 2 independent variables (attributes or features ), our model consists in finding a plane in 3D space that best fits the data. What happens when the number of attributes is equal to $n$?

In general case, our model can be represented as follows: $$y = \beta_0 + \beta_1 x_1 + ... + \beta_n x_p + \epsilon$$ 
where $x_i\in \mathrm{R}\ (i=\bar{1, p})$ are $p$ features, $\beta_j\in \mathrm{R}\ (j = \bar{0, p})$ are coefficients of the model, and $\epsilon\in\mathrm{R}^n$ is an error term.

Or in matrix form: $$\mathbf{y} = X\mathbf{\beta} + \mathbf{\epsilon}$$
where $n$ is the number of observations (samples), $p$ the number of features, $y\in \mathrm{R}^n, \mathbf{\beta} \in \mathrm{R}^{p+1}, X\in \mathrm{R}^{n\times(p+1)} \mathbf{\epsilon}\in \mathrm{R}^{n}$. The corresponding *normal equation* is given by: $X'\mathbf{y} = X'X\mathbf{\beta}$

**Assumptions on the error term $\mathbf{\epsilon} = [\epsilon_1, \epsilon_2, ..., \epsilon_n]^T \sim \mathcal{N}(0, \mathbf{\sigma}^2\mathbf{I})$ (independent, identically distributed normal errors)**:
1. Normality: errors follow a normal distributions $\epsilon_i \sim \mathcal{N}(0, \sigma^2)$
2. Independence: $\epsilon_i \sim \mathcal{N}(0, \sigma^2)$ independently, i.e. $Cov(\epsilon_i, \epsilon_j) = 0$ for $i\neq j$. Which also means that there is no correlation between errors. This validates standard error calculations.
3. Zero mean: $\mathrm{E}[\epsilon_i] = 0, \forall i$ ($\mathrm{E}[\mathbf{\mathbf{\epsilon}}] = 0_n$, zero vector) which implies that there is no systematic bias and ensures unbiased estimates $\mathrm{E}[\hat{\beta}] = \beta$
4. Constant variance (homoscedasticity): $Var(\epsilon_i) = \sigma^2 \mathbf{I}, \forall i$ where $\mathbf{I}$ is the identity matrix.



This error term represents errors in $y$ and natural randomness in the process. It embodies unobserved variables affecting $y$ and model specification errors.

*Note*: In machine learning, we often relax these assumptions. Deep learning, for instance, makes no distributional assumptions about errors. But understanding these classical assumptions helps us know when linear regression is appropriate and how to diagnose problems.

**Geometric Interpretation:**

In $(p+1)$-dimensional space, we fit a $p$-dimensional hyperplane through the data points. The regression finds the hyperplane that minimizes the sum of squared perpendicular distances. This plane $\hat{y} = X\hat{\beta}$ is the orthogonal projection of $y$ onto the space of $X$.

**OLS in general case:**

Our *objective* is to minimize $||\mathbf{y} - X\mathbf{\beta}||^2$

**Solution** (when $X'X$ is invertible) is given by: $$\mathbf{\hat{\beta}} = (X'X)^{-1}X'y$$ 

Let's apply this generalised method to our toy example.

1. Step 1: Let's add an intercept column to our features as the first column. At the end, we obtain a 9×2 matrix $X$ (9 observations, 2 parameters):

$$X = \left(\begin{matrix} 1 & 1.0 \\ 1 & 1.5 \\ 1 & 2.0 \\ 1 & 2.5 \\ 1 &  3.0 \\ 1 & 3.5 \\ 1 &  4.0\\ 1 &  4.5\\ 1 &  5.0 \end{matrix}\right)$$

We also have out output vector $y$:

$$y = \left(\begin{matrix} 70.56 \\ 67.68 \\ 80.88 \\ 82.32 \\ 84.00 \\ 90.00 \\ 93.60 \\ 105.36 \\ 109.92 \end{matrix}\right)$$

In [ ]:
X = np.column_stack([np.ones(len(data_x)), data_x])  # Add intercept
print(X)

2. Step 2: Calculate transpose $X^T$:

$$X^T = \left(\begin{matrix} 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\ 1.0 & 1.5 & 2.0 & 2.5 & 3.0 & 3.5 & 4.0 & 4.5 &  5.0 \end{matrix}\right)$$

In [ ]:
X.T

3. Step 3: Calculate $X^TX$:

$$X^TX = \left(\begin{matrix} 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\ 1.0 & 1.5 & 2.0 & 2.5 & 3.0 & 3.5 & 4.0 & 4.5 &  5.0 \end{matrix}\right)\left(\begin{matrix} 1 & 1.0 \\ 1 & 1.5 \\ 1 & 2.0 \\ 1 & 2.5 \\ 1 &  3.0 \\ 1 & 3.5 \\ 1 &  4.0\\ 1 &  4.5\\ 1 &  5.0 \end{matrix}\right)$$

Let's perform the calculation for each element:

$X^TX[1, 1] = 1\times 1 + 1\times 1 + 1\times 1 + 1\times 1 + 1\times 1 + 1\times 1 + 1\times 1 + 1\times 1 + 1\times 1 = 9$

$X^TX[1, 2] = 1\times 1.0 + 1\times 1.5 + 1\times 2.0 + 1\times 2.5 + 1\times 3.0 + 1\times 3.5 + 1\times 4.0 + 1\times 4.5 + 1\times 5.0 = 27.0$

$X^TX[2, 1] = 1.0 \times 1 + 1.5 \times 1 + 2.0 \times 1 + 2.5 \times 1 + 3.0 \times 1 + 3.5 \times 1 + 4.0 \times 1 + 4.5 \times 1 + 5.0 \times 1 = 27.0$

$X^TX[2, 2] = 1.0 \times 1.0 + 1.5 \times 1.5 + 2.0 \times 2.0 + 2.5 \times 2.5 + 3.0 \times 3.0 + 3.5 \times 3.5 + 4.0 \times 4.0 + 4.5 \times 4.5 + 5.0 \times 5.0 = 96.0$

$$X^TX = \left(\begin{matrix} 9.0 & 27 \\ 27.0 & 96.0 \end{matrix}\right)$$

In [ ]:
xtx = X.T @ X
print(xtx)


4. Step 4: Calculate the inverse $(X^TX)^{-1}$

For a 2x2 matrix $\left(\begin{matrix} a & b \\ c & d \end{matrix}\right)$, the inverse is given by $(1/det) \times \left(\begin{matrix} d & -b \\ -c & a \end{matrix}\right)$ where $det$ is the determinant of the matrix $X^TX$.

$$det(X^TX) = 9.0\times 96.0 - 27.0\times 27 = 864.0 - 729.0 = 135.0$$

$$(X^TX)^{-1} = \frac{1}{135.0} \times \left(\begin{matrix} 96.0 & -27 \\ -27.0 & 9.0 \end{matrix}\right) = \left(\begin{matrix} \frac{96.0}{135} & \frac{-27}{135} \\ \frac{-27.0}{135} & \frac{9.0}{135} \end{matrix}\right) = \left(\begin{matrix} 0.7111 & -0.2 \\ -0.2 & 0.0667 \end{matrix}\right)$$

Let's verify this is correct by checking $(X^TX)(X^TX)^{-1} = I$:

$$(X^TX)(X^TX)⁻¹ = \left(\begin{matrix} 9.0 & 27 \\ 27.0 & 96.0 \end{matrix}\right) \left(\begin{matrix} 0.7111 & -0.2 \\ -0.2 & 0.0667 \end{matrix}\right) =\left(\begin{matrix} 9\times0.7111 + 27\times(-0.2) & 9\times(-0.2) + 27\times 0.0667 \\ 27\times0.7111 + 96\times(-0.2) & 27\times(-0.2) + 96\times 0.0667 \end{matrix}\right) =$$

$$= \left(\begin{matrix} 6.4 - 5.4 & -1.8 + 1.8 \\ 19.2 - 19.2 & -5.4 + 6.4 \end{matrix}\right) =\left(\begin{matrix} 1 & 0 \\ 0 & 1 \end{matrix}\right)$$

In [ ]:
# calculate the inverse
xtx_inv = np.linalg.inv(xtx)
print(xtx_inv)

5. Step 5: Calculate $X^Ty$

$$X^Ty = \left(\begin{matrix} 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\ 1.0 & 1.5 & 2.0 & 2.5 & 3.0 & 3.5 & 4.0 & 4.5 &  5.0 \end{matrix}\right)\left(\begin{matrix} 70.56 \\ 67.68 \\ 80.88 \\ 82.32 \\ 84.00 \\ 90.00 \\ 93.60 \\ 105.36 \\ 109.92 \end{matrix}\right) =$$
$$\left(\begin{matrix} 1\times 70.56 + 1\times 67.68 + 1\times 80.88 + 1\times 82.32 + 1\times 84.00 + 1\times 90.00 + 1\times 93.60 + 1\times 105.36 + 1\times 109.92 \\ 1\times 70.56 + 1.5\times 67.68 + 2\times 80.88 + 2.5\times 82.32 + 3\times 84.00 + 3.5\times 90.00 + 4\times 93.60 + 4.5\times 105.36 + 5\times 109.92\end{matrix}\right) =$$
$$= \left(\begin{matrix} 784.32 \\ 70.56 + 101.52 + 161.76 + 205.80 + 252.00 + 315.00 + 374.40 + 474.12 + 549.60\end{matrix}\right) = \left(\begin{matrix} 784.32 \\ 2504.76\end{matrix}\right)$$

In [ ]:
xty = X.T @ data_y
print(xty)

6. Step 6: Calculate $\mathbf{\hat{\beta}} = (X'X)^{-1}X'y$

$$\mathbf{\hat{\beta}} = (X'X)^{-1}X'y = \left(\begin{matrix} 0.7111 & -0.2 \\ -0.2 & 0.0667 \end{matrix}\right)\left(\begin{matrix} 784.32 \\ 2504.76\end{matrix}\right)$$

$$=\left(\begin{matrix} 0.7111\times 784.32 + (-0.2)\times 2504.76 \\ -0.2 \times 784.32 +  0.0667 \times 2504.76 \end{matrix}\right) = \left(\begin{matrix} 557.73 - 500.952  \\ -156.864 +  167.0675 \end{matrix}\right) = \left(\begin{matrix} \mathbf{56.778}  \\ \mathbf{10.203} \end{matrix}\right) = \left(\begin{matrix} \mathbf{\beta_0}  \\ \mathbf{\beta_1} \end{matrix}\right) = \left(\begin{matrix} intercept  \\ slope \end{matrix}\right)$$

In [ ]:
b = xtx_inv @ xty
print(b)

The resulting regression equation:
$Height = 56.778 + 10.203 \times Age$

*Interpretation*:
- Intercept (56.778 cm): Expected height when $age = 0$ (theoretical baseline)
- Slope (10.203 cm/year): For each additional year of age, height increases by about 10.2 cm

## Maximum Likelihood Estimation (MLE) vs OLS
<a id="mle"></a>

Under the assumption that errors are normally distributed, MLE and OLS yield identical results. The key insight is that minimizing squared errors is equivalent to maximizing the likelihood of observing the data.

Our linear regression model has the following form: $y = X\beta + \epsilon$. This is a deterministic relationship plus an error term. To make distributional assumptions about the error term ε to enable statistical inference.

Assume errors are independent and normally distributed with zero mean and constant variance $\epsilon \sim \mathcal{N}(0, \sigma^2I)$, and are independent from $X$. 

According to the linear transformation property, if $Z \sim \mathcal{N}(\mu, \Sigma)$ then $aX + b \sim \mathcal{N}(a\mu + b, a^2\Sigma)$. In our case, since $\epsilon \sim \mathcal{N}(0, \sigma^2I)$, then $y = X\beta + \epsilon \sim \mathcal{N}(0 + X\beta, \sigma^2I) = \mathcal{N}(X\beta, \sigma^2I)$. This is true if we consider that $X$ is fixed and non-random, and is therefore treated like experimental conditions (*classical regression theory*). In this case, using our toy example, we would say "*Suppose we fix the ages at [1.0, 1.5, 2.0, ..., 5.0]. For these fixed ages, the heights vary randomly according to: $y \sim \mathcal{N}(X\beta, \sigma^2I)$*".

If we want to emphasise that observational data $X$ and $y$ are both random (*modern regression theory*), we rather use conditional notation: $y|X = X\beta + \epsilon|X = X\beta + \epsilon \sim \mathcal{N}(X\beta, \sigma^2I)$ which means "*Given that we observe specific values of $X$, the distribution of $y$ follows this normal distribution*". In terms of our example, we would say "*In practice, both age and height are random variables. But given that we observe specific ages, the conditional distribution of heights is:* $y|X \sim \mathcal{N}(X\beta, \sigma^2I)$".

Since $y|X \sim \mathcal{N}(X\beta, \sigma^2I)$, each individual observation follows: $y_i|x_i \sim \mathcal{N}(x_i^T\beta, \sigma^2)$, where:
* $x_i$ is the i-th row of $X$ (i-th observation's features)
* $x_i^T\beta = \beta_0 + \beta_1x_1 + ... + \beta_px_p = \mu_i$ is the expected value. Let's denote it with $\mu_i$.
* $\sigma^2$ is the variance (same for all observations). 

Thus, for a single observation $y_i$, the probability density function (PDF) is given by:
$$f(y_i|x_i, \beta, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(y_i - x_i^T\beta)^2}{2\sigma^2}}$$

Assume observations are independent given $X$. Under this key assumption, the joint density is the product of individual densities: 
$$f(y_1y_2...y_m|X, \beta, \sigma^2) = f(y_1|x_1, \beta, \sigma^2) \times f(y_2|x_2, \beta, \sigma^2) \times ... \times f(y_n|x_n, \beta, \sigma^2) = \prod_{i=1}^n(\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(y_i - x_i^T\beta)^2}{2\sigma^2}})$$

Now, considering our age-height data, the likelihood can be expressed as: "*Given these specific ages, what's the probability of observing these specific heights for different values of $\beta$ and $\sigma^2$*". So:

$$L(\beta, \sigma^2) = f(y|X, \beta, \sigma^2) = \prod_{i=1}^n(\frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(y_i - x_i^T\beta)^2}{2\sigma^2}})$$

We can factor out the constants:
$$L(\beta, \sigma^2) = \left(\frac{1}{\sqrt{2\pi\sigma^2}}\right)^n\prod_{i=1}^n(e^{-\frac{(y_i - x_i^T\beta)^2}{2\sigma^2}})$$

$$L(\beta, \sigma^2) = \left(\frac{1}{2\pi\sigma^2}\right)^{n/2}e^{\sum_{i=1}^n\left(-\frac{(y_i - x_i^T\beta)^2}{2\sigma^2}\right)} = \left(\frac{1}{2\pi\sigma^2}\right)^{n/2}e^{\left(\frac{-\sum_{i=1}^n(y_i - x_i^T\beta)^2}{2\sigma^2}\right)} = \left(2\pi\sigma^2\right)^{-n/2}e^{\left(\frac{-\sum_{i=1}^n(y_i - x_i^T\beta)^2}{2\sigma^2}\right)}$$

Recalling that the sum of squared error is given by $SSE = \sum_{i=1}^n (y_i = x_i^T\beta)^2$, we obtain: 

$$L(\beta, \sigma^2) = \left(2\pi\sigma^2\right)^{-n/2}e^{\left(\frac{-SSE}{2\sigma^2}\right)}$$

Taking the natural logarithm:

$$\mathcal{l}(\beta, \sigma^2) = \ln L(\beta, \sigma^2) = \ln\left(\left(2\pi\sigma^2\right)^{-n/2}\right) + \ln(e^{\left(\frac{-SSE}{2\sigma^2}\right)}) =$$
$$= -n/2 \ln(2\pi\sigma^2) + \left(\frac{-SSE}{2\sigma^2}\right) = -n/2\ln(2\pi) - n/2\ln(\sigma^2) - \frac{SSE}{2\sigma^2}$$
$$= -n/2\ln(2\pi) - n\ln(\sigma) - \frac{SSE}{2\sigma^2}$$

To find the MLE, we maximize $\mathcal{l}(\beta, \sigma^2)$ with respect to $\beta$ and $\sigma$:

$$\left\{\begin{aligned}\frac{\partial\mathcal{l}}{\partial\beta} = 0 \\\frac{\partial\mathcal{l}}{\partial\sigma} = 0\end{aligned}\right.$$

Since the only term containing $\beta$ is $- \frac{SSE}{2\sigma^2}$:

$$\frac{\partial\mathcal{l}}{\partial\beta} = \frac{\partial}{\partial \beta}\left(- \frac{SSE}{2\sigma^2}\right) = -\frac{1}{2\sigma^2}\frac{\partial SSE}{\partial\beta}$$

Since $SSE = (y-X\beta)^T(y-X\beta)$:
$$\frac{\partial SSE}{\partial\beta} = -2X^T(y - X\beta)$$

Then:
$$\frac{\partial\mathcal{l}}{\partial\beta} = -\frac{1}{2\sigma^2}(-2X^T(y - X\beta)) = 0$$

$$\frac{X^T(y - X\beta)}{\sigma^2} = 0$$

$$X^T(y - X\beta) = 0$$

$$X^Ty = X^TX\beta$$

$$\hat{\beta}_{MLE} = (X^TX)^{-1}X^Ty$$

which is exactly the OLS estimator ($\hat{\beta}_{MLE} = \hat{\beta}_{OLS}$).

**To sum up**: Under normal error assumptions, maximizing likelihood is equivalent to minimizing sum of squared errors.
From the log-likelihood: $\mathcal{l}(\beta, \sigma^2) = -n/2 log(2\pi\sigma^2) - SSE/(2\sigma^2)$

For fixed $\sigma^2$, maximizing $\mathcal{l}$ is equivalent to minimizing SSE, since:
- The first term doesn't depend on $\beta$
- The second term decreases as $SSE$ decreases

For $\sigma$, maximising the log-likelihood gives:

$$\frac{\partial\mathcal{l}}{\partial\sigma} = -\frac{n}{\sigma} + \frac{SSE}{\sigma^3} = 0$$
$$\frac{1}{\sigma}\left(\frac{SSE}{\sigma^2} - n\right) = 0$$
$$\frac{SSE}{\sigma^2} - n = 0$$
$$\frac{SSE}{\sigma^2} = n$$
$$\sigma^2 = \frac{SSE}{n}$$
$$\hat{\sigma}_{MLE} = \sqrt{\frac{SSE}{n}}$$

This differs slightly from the unbiased OLS estimator: $\hat{\sigma}_{OLS} = \sqrt{SSE/(n-p)}$ where $p$ is the number of parameters, i.e. the estimators differ by degrees of freedom.

Let's verify that with our example:

In [ ]:
# OLS solution
X = np.column_stack([np.ones(len(data_x)), data_x])
beta_ols = np.linalg.inv(X.T @ X) @ (X.T @ data_y)
# Calculate residuals and unbiased sigma
y_pred = X @ beta_ols
residuals = data_y - y_pred

n = len(data_y)
p = X.shape[1]
sse = np.sum(residuals**2)
# unbiased estimator
sigma_ols = np.sqrt(sse / (n - p))
print("OLS results")
print(f"intercept: {beta_ols[0]:.4f}")
print(f"scope: {beta_ols[1]:.4f}")
print(f"sigma: {sigma_ols:.4f}")

In [ ]:
from scipy.optimize import minimize

In [ ]:
# MLE solution
def negative_log_likelihood(params, X: np.array, y: np.array):
    """Negative log-likelihood for linear regression with normal errors

    Model: y_i = β₀ + β₁ * x_i + ε_i, where ε_i ~ N(0, σ²)
    
    Log-likelihood: ℓ(β₀, β₁, σ) = Σᵢ log(φ((y_i - β₀ - β₁*x_i)/σ)) - n*log(σ)
    where φ is the standard normal PDF
    
    Args:
        params: values of parameters beta and sigma
        X (np.array): features X
        y (np.array): vector y

    Returns:
        negative log-likelihood
    """
    beta0, beta1, log_sigma = params
    
    sigma = np.exp(log_sigma)
    # predicted values
    predictions = beta0 + beta1 * X
    # residuals
    residuals = y - predictions
    
    # Log-likelihood calculation
    # For normal distribution: log(L) = -n/2 * log(2π) - n*log(σ) - 1/(2σ²) * Σ(residuals²)
    n = len(y)
    log_likelihood = (-n/2 * np.log(2*np.pi) 
                     - n * np.log(sigma) 
                     - np.sum(residuals**2) / (2 * sigma**2))
    
    
    # Negative log-likelihood to minimize
    return -log_likelihood

In [ ]:
# Use OLS as initial guess
# initial_guess = [beta_ols[0], beta_ols[1], np.log(sigma_ols)]
initial_guess = [60, 10, 5]
# Minimize negative log-likelihood
result = minimize(negative_log_likelihood, initial_guess, args=(data_x, data_y), 
                     method='BFGS', options={'disp': False})

beta_0_mle, beta_1_mle, _ = result.x

# prediction MLE 
y_pred_mle = beta_0_mle + beta_1_mle * data_x 
residuals_mle = data_y - y_pred_mle 
sigma_mle = np.sqrt(np.mean(residuals_mle**2))

print("MLE results")
print(f"intercept: {beta_0_mle:.4f}")
print(f"slope: {beta_1_mle:.4f}")
print(f"sigma: {sigma_mle:.4f}")

Note that the intercept and the slope are the same.

## Confidence Intervals for Coefficients
<a id="confidence-intervals"></a>

**Confidence intervals** provide a range of plausible values for regression coefficients, expressing uncertainty in our parameter estimates. The coefficients are then given by: $$\hat{\beta}_i \pm t(\alpha/2, n-p-1) * SE(\hat{\beta}_i)$$
where:
- $\hat{\beta}_i$ denotes our estimated coefficients;
- $t(\alpha/2, n-p-1)$ denotes the critical $t$-value with $(n-p-1)$ degrees of freedom;
- $SE(\hat{\beta}_i) = \sqrt{\sigma^2 diag((X'X)^{-1})}$ denotes standard error of coefficient and where $\sigma^2 = SSE / (n-p-1)$ is the estimated error variance.

*Interpretation*: If we repeated this study many times, 95% of such intervals would contain the true coefficient value.

In [ ]:
# coefficients with confidence intervals
coeffs, se, ci_low, ci_high = calculate_confidence_intervals(data_x, data_y, confidence_level=0.95)
print(f"Intercept: {coeffs[0]:.3f} ± {se[0]:.3f}")
print(f"Slope: {coeffs[1]:.3f} ± {se[1]:.3f}")
print(f"95% CI for slope: [{ci_low[1]:.3f}, {ci_high[1]:.3f}]")

In [ ]:
# visualisation
coeffs, sigma, r2 = plot_regression_with_intervals(data_x, data_y)

On the above plot, we can interpret the red shaded area as follows: "We are 95% confident that the average height for children of age $x$ falls within this band". This area represents uncertainty in the fitted line itself.

Note that the area is narrower around the center of data (where $\bar{x}$ is).

As for the *prediction interval* (gray shaded area), we can interpret it as follows: "We are 95% confident that a single new observation at age $x$ will fall within this band". 

Note that it is always wider than confidence interval and accounts for both model uncertainty AND individual variation.

### $R^2$ ($R$-squared)
<a id="r-squared"></a>

> How much better my regression line is than another one? How much better is my regression line compared to just guessing the average every time?

**Intuition:**

In terms of our toy example, let's say that we are trying to predict children's heights, but we know nothing about their ages. Our best strategy would be to always guess the average height (say, 85 cm). We'd be wrong most of the time, with some kids much taller and others much shorter than your guess.

Now, if we ass age into our model to predict height, we would like to know what fraction of our "wrongness" we've eliminated by using the regression instead of just guessing the average.

A measure that can help us with this issue is called **coefficient of determination**, $R^2$:
- $R^2 = 0$: Your regression line is no better than always guessing the average
- $R^2 = 0.7$: Your regression eliminates 70% of the error you'd make by always guessing the average
- $R^2 = 1$: Perfect prediction - your line passes through every data point exactly

$R^2$ can be understood as the percentage of variation explained: If height varies wildly when you ignore age, but becomes much more predictable when you account for age, then $R^2$ will be high. 

<strong>Coefficient of determination</strong> $R^2$ is defined as: $$R^2 = 1 - \frac{SSE}{SST}$$ 
where: 
- $SSE$ denotes the sum of squared errors and is given by: $SSE = \sum_{i=1}^n(y_i - \hat{y}_i)^2$. It measures how much variation remains unexplained after fitting your regression line. This is the residual error your model still makes.
- $SST$ denotes the total sum of squares and is given by: $SST = \sum_{i=1}^n(y_i - \bar{y})^2$. It measures the total variation in the data around the mean. This represents how much the data would "spread out" if you knew nothing except the average.
- $\frac{SSE}{SST}$ this ratio tells you what proportion of the original variation you failed to explain with your model.

An alternative formulation is: $R^2 = \frac{SSR}{SST}$ where $SSR = SST - SSE$ denotes the sum of squares due to regression.

$R^2$ ranges between 0 and 1 (or 0-100%).

*Warning!*: High $R^2$ doesn't imply causation - it only measures linear association.

Note that $R^2$ is sample dependent. It tends to increase as you add more variables, even if they're irrelevant (leading to adjusted $R^2$ for multiple regression).

$R^2$ equals the square of the correlation coefficient ($r$). This connects the geometric notion of how tightly data clusters around a line (correlation) with the variance-explained interpretation ($R^2$).

### Residual Plot
<a id="residual-plot"></a>

If your model is correctly specified, residuals should look like pure random noise. Any patterns in residuals indicate model inadequacy. This is because: if $y = f(x) + \epsilon$ where $\epsilon \sim \mathcal(0, \sigma^2)$ then $residuals = y - \hat{y} \approx \epsilon$ (random noise). Patterns in residuals imply that the function $f(x)$ is not linear.

Residual plots are the primary diagnostic tool for regression. It plots residuals ($\hat{e} = y - \hat{y}$) against fitted values or predictor variables. It allows to check regression assumptions:

- Linearity: Residuals should be randomly scattered
- Homoscedasticity: Constant variance across fitted values
- Independence: No patterns in residuals

In [ ]:
# Example 0: Data that respects the assumptions
np.random.seed(42) # fix random seed to reproducibility

ex0_x = np.random.rand(400) * 10
ex0_error = np.random.rand(400) * 2
ex0_y_true = 2 * ex0_x + 1 + ex0_error

# visualisation
create_static_visualization(ex0_x, ex0_y_true, x_label_text="x", y_label_text="y = 2x + 1 + eps")
print("Random scatter around zero. No visible patterns of residuals")

### MSE and RMSE
<a id="mse-rmse"></a>

We would like to further assess the quality of our predictions. 

> How wrong are our predictions, on average?

*Intuition:* 

You are making a prediction of the height given the age = 3:
- True height: 84.00
- Predicted height: 87.147
- Error: 3.147

Now, imagine that you make many such predictions. We are interested in summarising the typical magnitude of the errors across all predictions. 

Two measures can be used: **MSE** (Mean Squared Error) and **RMSE** (Root Mean Squared Error)

$$MSE = \frac{\sum_{i=1}^n(y_i - \hat{y_i})^2}{n} = \frac{\sum_{i=1}^n e_i^2}{n}$$
where:
- $n$ is the number of observations
- $y_i$ is the true value for observation $i$
- $\hat{y_i}$ is the predicted value for observation $i$
- $e_i = (y_i - \hat{y_i})$ is residual (prediction error) for observation $i$

$$RMSE = \sqrt{MSE} = \sqrt{\frac{\sum_{i=1}^n(y_i - \hat{y_i})^2}{n}}$$

Note that one of the advantages to use of squared errors is that this penalises large errors more heavily or in other words, quadratic penalty makes the model focus on reducing large mistakes. Example: error of 2 contributes 4 to MSE, while error of 4 contributes 16 to MSE (i.e. 4x(the penalty for 2)).

| | MSE | RMSE |
|--|:----:|:------:|
| Formula | $$MSE = \frac{\sum_{i=1}^n(y_i - \hat{y_i})^2}{n} = \frac{\sum_{i=1}^n e_i^2}{n}$$ |  $$RMSE = \sqrt{MSE} = \sqrt{\frac{\sum_{i=1}^n(y_i - \hat{y_i})^2}{n}}$$| 
| Units | squared units of the target variable | same units as the target variable|
| Interpretation | $\Rightarrow$ direct interpretation is challenging | standard deviation of prediction errors </br>(e.g. RMSE = $15,000 means "*typical prediction error is about $15,000*")</br> Roughly 68% of predictions are within ±1 RMSE of true values (assuming normal error distribution)| 
| Range | $MSE \geq 0$ | $RMSE \geq 0$ |
|Perfect prediction | $MSE =0$ | $RMSE = 0$|
| Scale sensitivity | increases quadratically with the scale of data | increases linearly with the scale of data|
| Sensitivity to outliers | yes | yes |
|When to use | - Mathematical derivations: MSE has nicer mathematical properties </br> - Optimization: Algorithms often minimize MSE directly </br> - Comparing model complexity: In regularization, we add penalties to MSE </br> - Computational efficiency: Avoiding square root calculation | - Reporting results: RMSE is more interpretable </br> - Setting expectations: "Typical error is about X units" </br> - Comparing models on the same scale: RMSE maintains original units </br> - Communicating with non-technical stakeholders|

Note: MSE and RMSE are **necessary but not sufficient** for model evaluation. Always complement them with residual analysis, validation curves, and domain-specific metrics to get the complete picture of model performance.

When it comes to **expected error over all possible training sets**, in other words expected MSE over all possible training sets $\mathrm{E}[(y_0 - \hat{f}(x_0))^2]$, it can be represented as $\mathrm{E}[(y_0 - \hat{f}(x_0))^2] = Bias^2 + Variance + Noise$, i.e. using *bias-variance decomposition*. Let's demonstrate that.

Let $y = f(x) + \epsilon$ be a true function with $\epsilon \sim \mathcal{N(0, \sigma^2)}$. Let $\hat{f}(x)$ be an estimator trained on dataset $D$. And let $(x_0, y_0)$ where $y_0 = f(x_0) + \epsilon$ be a new point (out of sample $D$). What's the expected squared error $\mathrm{E}[(y_0 - \hat{f}(x_0))^2]$?

1. Substitute $y_0 = f(x_0) + \epsilon_0$:
$$\mathrm{E}[(y_0 - \hat{f}(x_0))^2] = \mathrm{E}[(f(x_0) + \epsilon_0 - \hat{f}(x_0))^2]$$

2.  Add and substract $\mathrm{E}[\hat{f}(x_0)]$ (the expected prediction):
$$= \mathrm{E}[(f(x_0) + \epsilon_0 - \mathrm{E}[\hat{f}(x_0)] + \mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0))^2]$$

3. Rearrange terms
$$= \mathrm{E}[(\underbrace{(f(x_0) - \mathrm{E}[\hat{f}(x_0)])}_{a} + \underbrace{(\epsilon_0)}_{b}  + \underbrace{(\mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0))}_{c})^2]$$

4. Expand the square $(a + b + c)^2 = a^2 + b^2 + c^2 + 2ab + 2ac + 2bc$
$$= \mathrm{E}[((f(x_0) - \mathrm{E}[\hat{f}(x_0)]))^2] + \mathrm{E}[(\epsilon_0)^2] + \mathrm{E}[(\mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0))^2] + \text{cross\_terms}$$

5. Evaluate cross terms
- $\mathrm{E}[\underbrace{\epsilon_0}_{noise}\cdot\underbrace{(f(x_0) - \mathrm{E}[\hat{f}(x_0)])}_{bias}] = 0$ as noise is independent of deterministic bias
- $\mathrm{E}[\underbrace{\epsilon_0}_{noise}\cdot\underbrace{(\mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0))}_{\text{deviation from expected prediction}}] = \mathrm{E}[\epsilon_0]\cdot\mathrm{E}[\mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0)] = 0$ as test point noise $\epsilon_0$ is independent of training data randomness and $\mathrm{\epsilon_0} = 0 \Rightarrow \mathrm{E}[\epsilon_0 \cdot \text{anything\_independent\_of\_}\epsilon_0] = 0$ 
- $\mathrm{E}[\underbrace{(f(x_0) - \mathrm{E}[\hat{f}(x_0)])}_{bias}\cdot\underbrace{(\mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0))}_{\text{deviation from expected prediction}}] = 0$ as bias is independent of the "variability part" that becomes the variance when squared

6. Final decomposition
$$= \mathrm{E}[((f(x_0) - \mathrm{E}[\hat{f}(x_0)]))^2] + \sigma^2 + \mathrm{E}[(\mathrm{E}[\hat{f}(x_0)] - \hat{f}(x_0))^2] + 0 =$$
$$= Bias^2(\hat{f}(x_0)) + \text{Irreducible Error} + Variance(\hat{f}(x_0))$$

Practical Meaning:

- Bias²: How far off our average prediction is from truth
- Variance: How much our predictions vary across different training sets
- Irreducible Error: Noise that can't be predicted

## Linear Regression from Scratch
<a id="linreg-from-scratch"></a>

<div class="alert-exercise">
<h5> QUESTION 2:</h5> Write linear regression model in general form from scratch using Python. The function should return estimated coefficients: β = (X'X)^-1 X'y. Test on our toy example.

```
def linear_regression_scratch(X: np.array, y: np.array) -> np.array:
    """Implements linear regression model using OLS and returns the coefficients.

    Args:
        X (np.array): array of observed feature values
        y (np.array): array of observed outcomes

    Returns:
        np.array: estimated coefficients
    """
```

Some hints:
- Mind to add the intercept column to X. You can initialise it with ones. To do so, you can use [numpy.column_stack()](https://numpy.org/doc/stable/reference/generated/numpy.column_stack.html)
- For matrix multiplication, you can use a dedicated operator [`@`](https://peps.python.org/pep-0465/).
- To transpose a matrix, you can use the property [matrix.T](https://numpy.org/doc/2.3/reference/generated/numpy.matrix.T.html). Alternatively, you can use [numpy.transpose()](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)
- To invert a matrix, you can use [numpy.linalg.inv()](https://numpy.org/doc/2.0/reference/generated/numpy.linalg.inv.html) 
</div>

In [ ]:
# ANSWER
def linear_regression_scratch(X: np.array, y: np.array) -> np.array:
    """Implements linear regression model using OLS and returns the coefficients.

    Args:
        X (np.array): array of observed feature values
        y (np.array): array of observed outcomes

    Returns:
        np.array: estimated coefficients
    """
    
    pass
    

In [ ]:
# ANSWER
# test on the toy data


<div class="alert-exercise">
<h5> QUESTION 3:</h5> Calculate predicted values for our toy example.
</div>

In [ ]:
# ANSWER


<div class="alert-exercise">
<h5> QUESTION 4:</h5> Write a function that calculates the residuals and plots the Residual Plot. Apply the function to our toy example.

```
def calculate_residuals(y_pred: np.array, x: np.array, y_true: np.array) -> np.array:
    """
    Calculates residuals of the model and plot Residual Plot.
    
    Args:
        y_pred (np.array): predicted values
        x (np.array): array of observed feature values
        y_true (np.array): array of observed outcomes

    Returns:
        np.array: residuals (error = y_true - y_pred)
    """
```
</div>

In [ ]:
# ANSWER 
def calculate_residuals(y_pred: np.array, x: np.array, y_true: np.array) -> np.array:
    """
    Calculates residuals of the model and plot Residual Plot.
    
    Args:
        y_pred (np.array): predicted values
        x (np.array): array of observed feature values
        y_true (np.array): array of observed outcomes

    Returns:
        np.array: residuals (error = y_true - y_pred)
    """
    pass

In [ ]:
# ANSWER


<div class="alert-exercise">
<h5> QUESTION 5:</h5> Write a function that calculates SSE, MSE, RNSE and R^2. Test on our toy example.

```
def calculate_perf_stats(y_observed: np.array, y_pred: np.array) -> tuple[float, float, float, float]:
    """Calculates statistics (R-squared, MSE, RMSE, SSE) given observed and predicted values.

    Args:
        y_observed (np.array): observed values
        y_pred (np.array): predicted values
    
    Return:
        R-squared, MSE, RMSE, SSE
    """
```
</div>

In [ ]:
# ANSWER
def calculate_perf_stats(y_observed: np.array, y_pred: np.array) -> tuple[float, float, float, float]:
    """Calculates statistics (R-squared, MSE, RMSE, SSE) given observed and predicted values.

    Args:
        y_observed (np.array): observed values
        y_pred (np.array): predicted values
    
    Return:
        R-squared, MSE, RMSE, SSE
    """
    pass

In [ ]:
# ANSWER


## Linear Regression in Python
<a id="linreg-python"></a>

### Using `scipy.stats.lingress`

The first option consists in using [`scipy.stats.lingress`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html) function that calculates a linear least-squares regression for two sets of measurements. 

In [ ]:
from scipy.stats import linregress # linear regression

In [ ]:
# linear regression 
slope, intercept, r_value, p_value, std_err = linregress(data_x, data_y)

# create a regression line
reg_line = slope * data_x + intercept

# print the results
print(f"Regression line: regression_line = {slope} * x + {intercept}")
print(f"Slope: {slope}")
print(f"Intercept: {intercept}")
print(f"R-value (Pearson correlation coefficient): {r_value}")
print(f"R-squared (determination coefficient): {r_value**2}")
print(f"Standard error of the estimated slope (gradient): {std_err}")
print(f"p-value (H0: the slope is zero): {p_value}")

# show the data with the regression line
plt.scatter(data_x, data_y, label='Observed data')
plt.plot(data_x, reg_line, color='red', label='Regression line')
plt.legend()
plt.title('Linear regression with scipy.stats.lingress')
plt.xlabel('age')
plt.ylabel('height')
plt.show()

### Using `sklearn.linear_model.LinearRegression`

Alternatively, we can use [sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) that fits a linear model with coefficients w = (w1, …, wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

In [ ]:
from sklearn.linear_model import LinearRegression # linear regression

In [ ]:
# create a model object
lin_model = LinearRegression()
# fit the data
lin_model.fit(data_x.reshape(-1, 1), data_y)
# display the parameters of the model
slope_sk = lin_model.coef_
intercept_sk = lin_model.intercept_
r_squared_sk = lin_model.score(data_x.reshape(-1, 1), data_y)
print(f"Linear regression equation: f_sk = {slope_sk[0]}age + {intercept_sk}")
print(f"Determination coefficient: {r_squared_sk}")


To predict the value using this model, we can use `predict()` method:

In [ ]:
# prediction
lin_model.predict(np.array([[2.2]]))

## When Regression Breaks Down
<a id="breaks-down"></a>

Regression model breaks down when the statistical assumptions are violated. 
1. Linearity violation
2. Independence violation
3. Homoscedasticity violation
4. Normality violation

Let's explore that with examples:

In [ ]:
# Example 0: Data that respects the assumptions
np.random.seed(42) # fix random seed to reproducibility

ex0_x = np.random.rand(400) * 10
ex0_error = np.random.rand(400) * 2
ex0_y_true = 2 * ex0_x + 1 + ex0_error

# visualisation
create_static_visualization(ex0_x, ex0_y_true, x_label_text="x", y_label_text="y = 2x + 1 + eps")
print("Random scatter around zero. No visible patterns of residuals")

In [ ]:
# Example 1 (Linearity Violation): Quadratic relationship fitted with linear model
ex1_x = np.linspace(0, 10, 100)
ex1_y = ex1_x**2 + np.random.normal(0, 5, 100)

# visualisation
create_static_visualization(ex1_x, ex1_y, x_label_text="x", y_label_text="y = x^2 + eps")
print("We can see a curved residual pattern shown by this linear fit.")

In [ ]:
# Example 2 (Independence Violation): Time series with autocorrelation
ex2_t = np.arange(400)
ex2_error = np.cumsum(np.random.normal(0, 1, 400))  # Autocorrelated errors
ex2_y = 2 + 3*ex2_t + ex2_error

# visualisation
create_static_visualization(ex2_t, ex2_y, x_label_text="t", y_label_text="y = 2 + 3*t + autocor.error")
print("Residuals will show patterns over time")

In [ ]:
# Example 3 (Homoscedasticity Violation): Variance increases with x
ex3_x = np.linspace(1, 10, 400)
ex3_y = 2 + 3*ex3_x + np.random.normal(0, ex3_x, 400)  # Variance proportional to X

# visualisation
create_static_visualization(ex3_x, ex3_y, x_label_text="x", y_label_text="y = 2 + 3*x + error.increass.var")
print("Residual plot  shows fan shape")

In [ ]:
# Example 4 (Normality Violation): Heavy-tailed errors
ex4_x = np.linspace(0, 10, 400)
ex4_y = 2 + 3*ex4_x + np.random.laplace(0, 1, 400)  # Laplace distribution (heavy tails)
ex5_y = 2 + 3*ex4_x + (np.random.exponential(2, 400) - 2)  # exponential distribution (right-skewed)

create_static_visualization(ex4_x, ex4_y, x_label_text="x", y_label_text="y = 2 + 3*x + laplace.error")
print("In comparison to normal errors, errors following Laplace distribution create a wider spread pattern in residual plot")

create_static_visualization(ex4_x, ex5_y, x_label_text="x", y_label_text="y = 2 + 3*x + exp.error")
print("In comparison to normal errors, errors following exponential distribution (right-skewed) create more residuals on one side of zero than the other")

A further analysis with *Q-Q plot* (Quantile-Quantile plot) can be performed to check normality violations. 

Q-Q plots (Quantile-Quantile plots) compare the quantiles of your data against the quantiles of a theoretical distribution (usually normal).

**Interpretation:** 

On X-axis, there are theoretical quantiles (what you'd expect from a normal distribution) and on Y-axis, there are sample quantiles (actual values from your data, ordered). A reference line (perfect normal distribution) is also usually plotted. 

In case of "perfect normality", the points lie exactly on the diagonal reference line, which means that data quantiles match theoretical normal quantiles perfectly. Note that random scatter around the line is acceptable.

> How to use it for decision making for regression?

- Acceptable: Points roughly follow line with random scatter
- Borderline: Slight systematic deviations but overall linear trend
- Problematic: Strong S-curves (points curve away or toward line at ends), severe skewness (points below line on left, above on right), or multiple modes (steps or multiple curves)

One way to plot it is using [`scipy.stats.probplot`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html):

In [ ]:
from scipy.stats import probplot # quantiles for a probability plot

In [ ]:
ex4_residuals = (2 + 3*ex4_x) - ex4_y
fig, ax = plt.subplots(1, 1, figsize=(7, 6))
probplot(ex4_residuals, dist="norm", plot=ax) # plot parameter should be set for visualisation
print("Q-Q plot shows deviations from normal line")

## Polynomial Features
<a id="polynomial-features"></a>

By now, we have explored linear terms. But sometimes, it's not enough. 

> Why do we need non-linear terms?

- Real relationships are rarely linear
- Linear models underfit curved data
- Poor predictions and $R^2$

Let's consider the following examples revealing the limitations of linear terms:


In [ ]:
# problems revealed
fig_scenarios, mod_res = get_scenarios()

To overcome these limitations, we can use **polynomial terms**: $x, x^2, x^3$. Our model then becomes: $y = \beta_0 + \beta_1 x^1 + \beta_2 x^2 + ... \beta_p x^p$. Note that the model is still "linear" in parameters. As a general rule, the use of higher degree polynomials imply a better fit on the seen (train) data. 

In [ ]:
# polynomial solution
fig_poly = get_poly_fit()

But there is a high risk of **overfitting**!

In [ ]:
# overfit example
get_overfit_example()


We may note that training scores, both $R^2$ and $MSE$ improve with the increase of the polynomial degree. However, this performance is not maintained on the test data, demonstrating bad generalisation. 

On the above plot, we don't really evaluate and show variance (how much predictions change across different training sets) evolution as it requires multiple training sets.

Or consider the following example: 
Our true underlying function that we want to learn is $y = 1.5 * x^2 + 0.5 * x + 0.3$. See a green line on the top left plot. We generate a small dataset of 30 samples (observations) by adding some noise which follows Normal distribution.

![](img/overfitting.png)

As a general rule, we talk about underfitting/overfitting in the following contexts:
- High bias, low variance $\Rightarrow$ underfitting (e.g. degree 1)
- Low bias, low variance $\Rightarrow$ optimal balance (e.g. degree 3-4)
- Low bias, high variance $\Rightarrow$ overfitting (e.g. degree 5+)

To create polynomial features in Python, you can use [sklearn.preprocessing.PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). Generate a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to the specified degree. Thus, for the input of the form `[a, b]`, the degree-2 polynomial features are `[1, a, b, a^2, ab, b^2]`.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# create polynomial features of a given degree
degree = 2
poly_features = PolynomialFeatures(degree=degree)
# transform our toy example features
poly_features.fit_transform(data_x.reshape(-1, 1))

In the example above, we note that the first column is a column of ones. It corresponds to an intercept term in a linear model. As there is only one feature in `data_x`, then the second column corresponds to the values of features themselves. And the third column corresponds to the degree-2 values.

print(f"\n🎯 KEY LEARNING OBJECTIVES:")
print(f"1. Understand when linear models are insufficient")
print(f"2. Learn how polynomial features extend linear regression")
print(f"3. Recognize and prevent overfitting with high-degree polynomials")
print(f"4. See how regularization solves polynomial overfitting")
print(f"5. Understand interaction terms and feature explosion")


print(f"\n⚖️ BIAS-VARIANCE INSIGHTS:")
print(f"• Degree 1: High bias, low variance (underfitting)")
print(f"• Degree 2-3: Often optimal balance")
print(f"• Degree 5+: Low bias, high variance (overfitting)")
print(f"• Regularization: Manages variance while keeping flexibility")

print(f"\n🚀 BRIDGE TO ADVANCED TOPICS:")
print(f"• Neural Networks: Learned polynomial-like features")
print(f"• Kernel Methods: Implicit infinite-degree polynomials")  
print(f"• Splines: Piecewise polynomials")
print(f"• Feature Engineering: Domain-specific transformations")


## Regularisation in Linear Regression
<a id="regularisation"></a>

In standard linear regression, we minimize the least squares loss:
$L(\beta) = ||y - X\beta||^2$
This works well when we have more observations than features (n > p) and our model isn't too complex. However, problems arise when:

- High dimensionality: $p \approx n$ or $p > n$ (common in AI applications)
- Multicollinearity: Features are highly correlated
- Limited data: Small sample sizes relative to model complexity
- Noise: We want to prevent fitting to random fluctuations

The result is **overfitting**: perfect fit to training data but poor generalization to new data.

Regularization adds a penalty term to our objective function that constrains model complexity:
$$L_{regularized}(\beta) = ||y - X\beta||^2 + \lambda·Penalty(\beta)$$
where:
- $\lambda$ is a regularization parameter controlling penalty strength
- $Penalty(\beta)$ is a function that penalizes complex models

> What penalty can be used?

### Ridge Regression (L2 Regularisation)

Objective function: 

$$L_{Ridge}(\beta) = ||y - X\beta||^2 + \lambda·||\beta||_2^2$$

where the penalty term is given by: $||\beta||_2^2 = \beta_1^2 + \beta_2^2 + ... + \beta_p^2$

Key Properties:

- Shrinkage: Pulls coefficients toward zero proportionally
- Stability: Always has a unique solution, even when X'X is singular
- Grouping effect: Correlated features get similar coefficients
- No feature selection: Coefficients approach zero but never become exactly zero

Solution is then given by:

$$\hat{\beta}_{Ridge} = (X'X + \lambda I)^{-1}X'y$$

*Geometric interpretation*: The penalty creates a circular constraint region around the origin. The solution occurs where the loss function contours first touch this circle.

When to use:

- Many relevant features
- Features are correlated
- Prediction accuracy is primary goal
- Interpretability is less important

### Lasso Regression (L1 Regularisation)

Objective Function:
$$L_{Lasso}(\beta) = ||y - X\beta||^2 + \lambda||\beta||_1$$
where the penalty term is given by: $||\beta||_1 = |\beta_1| + |\beta_2| + ... + |\beta_p|$

Key Properties:

- Sparsity: Produces exactly zero coefficients (automatic feature selection)
- Non-differentiable: Requires specialized optimization algorithms
- Instability: Can arbitrarily select one feature from a group of correlated features
- Interpretability: Simpler models with fewer features

Solution is then given by:

$$\hat{\beta}_{Lasso} = (X'X + \lambda I)^{-1}X'y$$

*Geometric interpretation*: The L1 penalty creates a diamond-shaped constraint region. The sharp corners make it likely that the optimal solution will have some coefficients exactly at zero.

When to use:

- Feature selection is desired
- Believe many features are irrelevant
- Want interpretable models
- Features are not highly correlated 

### Elastic Net
Objective Function:
$$L_{ElasticNet}(\beta) = ||y - X\beta||^2 + λ_1||\beta||_1 + λ_2||\beta||_2^2$$

Often parameterized as:
$$L_{ElasticNet}(\beta) = ||y - X\beta||^2 + λ[\alpha||\beta||_1 + (1-α)||\beta||_2^2]$$
where $\alpha \in [0,1]$ controls the mix between L1 and L2 penalties.

Advantages:

- Feature selection (from L1) + Stability (from L2)
- Handles correlated features better than pure Lasso
- Flexible: Can tune the balance between sparsity and grouping

When to use:

- High-dimensional data with correlated features
- Want both feature selection and stability
- Examples: Genomic data, text analysis, image processing

> How bias-variance tradeoff is affected by regularisation?

Regularization fundamentally manages the bias-variance tradeoff:

|Underularised Regression | Regularised Regression|
|---|---|
|Low bias (if model is correctly specified)|Introduces bias (coefficients shrunk toward zero)|
|High variance (especially with limited data)|Reduces variance (more stable predictions)|
||Often lower total error: $\mathrm{E}[(\hat{y} - y)^2] = Bias^2 + Variance + Noise$|

</br>

> How to select the regularisation parameter $\lambda$?

The choice of $\lambda$ is critical and usually done through **cross-validation**:
1. Split data into $k$ folds
2. For each $\lambda$ value, train on $k-1$ folds and validate on the remaining fold
3. Average validation error across all folds
4. Select $\lambda$ that minimizes average validation error

To visually determine $\lambda$, you can plot validation curves, i.e. training and validation error vs. $\lambda$.

**Practical implementation tip:** Always standardize features before applying regularization, since the penalty treats all coefficients equally: $X_{scaled} = \frac{X - \mu}{\sigma}$.

## Model Selection Criteria: AIC and BIC
<a id="aic-bic"></a>

Model selection criteria help us balance two competing goals:
- *Goodness of fit*: How well does the model explain the observed data?
- *Model complexity*: How many parameters does the model use?

It is like finding the "sweet spot" between underfitting (too simple) and overfitting (too complex). These criteria penalize models for being complex while rewarding them for fitting the data well.

|| Akaike Information Criterion (AIC) | Bayesian Information Criterion (BIC)|
| ---|------ | ------ |
|Definition| $$AIC = 2k - 2ln(L)$$- $k$ = number of parameters in the model</br>- $L$ = max likelihood of the model</br>- $ln(L)$ = log-likelihood of the model | $$BIC = k\cdot ln(n) - 2ln(L)$$- $k$ = number of parameters</br>- $n$ = sample size</br>- $L$ = maximum likelihood</br>- $ln(L)$ = log-likelihood |
|Key properties |1. *Asymptotic efficiency*: Selects the model that minimizes prediction error as sample size approaches infinity </br>2. *Consistent selection*: Among nested models, tends to select the true model with high probability </br>3. *Relative measure*: Only meaningful when comparing models; absolute values don't matter</br>4. *Sample size independent penalty*: The penalty term ($2k$) doesn't depend on sample size |1. *Bayesian foundation*: Derived from Bayesian model comparison principles </br>2. *Consistent*: Selects the true model with probability approaching 1 as $n \rightarrow \infty$</br>3. *Sample size dependent penalty*: Penalty increases with sample size $(k\cdot ln(n))$</br>4. *More conservative*: Generally penalizes complexity more heavily than AIC, especially for larger samples|
|Advantages|- Well-suited for prediction-focused model selection</br>- Performs well in small samples</br>- Less likely to underfit</br>- Solid theoretical foundation in information theory|- Strong theoretical foundation in Bayesian statistics</br>- More conservative, reducing overfitting risk</br>- Consistent model selection (selects true model asymptotically)</br>- Adapts penalty based on sample size|
|Limitations|- May overfit in large samples<br>- Can select overly complex models when the true model is among candidates</br>- Doesn't account for sample size in penalty term|- May underfit, especially in small samples</br>- Can be too conservative for prediction tasks</br>- Assumes one of the candidate models is the "true" model</br>- Less suitable when the focus is purely predictive|
|When to use |- Primary goal is prediction accuracy</br>- Working with small to moderate sample sizes</br>- You want to err on the side of including potentially relevant variables</br>- The focus is on finding the best approximating model rather than the "true" model</br>- Cross-validation is not feasible|- Primary goal is identifying the true underlying model</br>- Working with large sample sizes</br>- You want to avoid overfitting</br>- Parsimony is particularly important (note: the penalty terms embody parsimony by making complex models "pay" for additional parameters)</br>- You're working within a Bayesian framework</br>- The candidate models include the true model|

**Practical Recommendations**

1. Use both criteria: Compare results from both AIC and BIC to understand the trade-off between fit and complexity
2. Consider sample size: In small samples ($n < 40$), AIC may be preferable. In large samples ($n > 100$), BIC's stronger penalty may be more appropriate
3. Domain knowledge: Don't rely solely on statistical criteria; incorporate subject matter expertise
4. Cross-validation: When possible, complement AIC/BIC with cross-validation for more robust model selection
5. Model diagnostics: Always check residuals and other diagnostics after selection
6. Uncertainty: If models have similar AIC/BIC values, consider model averaging or report uncertainty in model selection

As a standard practice, normal likelihood is used (most software does this automatically). But if you encounter other type of error distribution, use it. You can start with normal, then investigate if results seem unreasonable. 

|When normal likelihood is reasonable|When to consider alternatives|
|------|-----|
|- Residuals approximately normal (check with QQ plots, normality tests)</br>- No severe outliers</br>- Primary goal is prediction (normal likelihood often works well even with non-normal errors)</br>- Large samples (Central Limit Theorem makes normal approximation better)|- Heavy tails/outliers → t-distribution likelihood</br>- Binary outcomes → Bernoulli likelihood (logistic regression)</br>- Count data → Poisson likelihood</br>- Positive continuous data → Gamma likelihood</br>- Skewed data → Consider transformations or appropriate GLM (Generalized Linear Models)|

As discussed in the section [MLE vs OLS](#maximum-likelihood-estimation-mle-vs-ols), the log-likelihood for a normal distribution is given by:


<div class="alert-exercise">
<h5> QUESTION 6:</h5> Write a function that calculates AIC, BIC, log-likelihood, RSS, $\sigma^2$.

```
def calculate_aic_bic(y_true, y_pred, k, n):
    """
    Calculate AIC and BIC given predictions and model complexity
    
    Args:
        y_true: actual values
        y_pred: predicted values
        k: number of parameters (including intercept)
        n: sample size
    
    Returns:
        AIC, BIC, log-likelihood, residual sum of squares (RSS), sigma2
    """
```
</div>

In [ ]:
# ANSWER
def calculate_aic_bic(y_true, y_pred, k, n):
    """
    Calculate AIC and BIC given predictions and model complexity
    
    Args:
        y_true: actual values
        y_pred: predicted values
        k: number of parameters (including intercept)
        n: sample size
    
    Returns:
        AIC, BIC, log-likelihood, residual sum of squares (RSS), sigma2
    """
    pass

In [ ]:
def fit_polynomial_model(x, y, degree):
    """Fit polynomial model and return predictions and parameters"""
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=True)
    x_poly = poly_features.fit_transform(x.reshape(-1, 1))
    
    # Fit model using normal equations (more numerically stable for small datasets)
    coefficients = np.linalg.lstsq(x_poly, y, rcond=None)[0]
    y_pred = x_poly @ coefficients
    
    k = len(coefficients)  # number of parameters
    
    return y_pred, coefficients, k, x_poly

In [ ]:
# Polynomial Model Comparison
models_poly = {}
results_poly = []

# Fit polynomial models of different degrees
for degree in [1, 2, 3]:
    model_name = f"Polynomial Degree {degree}"
    
    # Fit model
    y_pred, coeffs, k, X_poly = fit_polynomial_model(data_x, data_y, degree)
    
    # Calculate AIC and BIC
    aic, bic, log_lik, rss, sigma2 = calculate_aic_bic(data_y, y_pred, k, n)
    
    # Store results
    models_poly[model_name] = {
        'degree': degree,
        'coefficients': coeffs,
        'y_pred': y_pred,
        'k': k,
        'aic': aic,
        'bic': bic,
        'log_likelihood': log_lik,
        'rss': rss,
        'r_squared': 1 - rss / np.sum((data_y - np.mean(data_y))**2)
    }
    
    results_poly.append({
        'Model': model_name,
        'Degree': degree,
        'Parameters (k)': k,
        'Log-Likelihood': log_lik,
        'RSS': rss,
        'R²': 1 - rss / np.sum((data_y - np.mean(data_y))**2),
        'AIC': aic,
        'BIC': bic
    })

# Create results DataFrame
df_poly = pd.DataFrame(results_poly)
print("\nPolynomial Model Results:")
print(df_poly.round(3))

# Calculate AIC and BIC differences
df_poly['ΔAIC'] = df_poly['AIC'] - df_poly['AIC'].min()
df_poly['ΔBIC'] = df_poly['BIC'] - df_poly['BIC'].min()

print("\nModel Comparison (Δ = difference from best model):")
print(df_poly[['Model', 'Parameters (k)', 'AIC', 'ΔAIC', 'BIC', 'ΔBIC']].round(3))

# Calculate Akaike weights
delta_aic = df_poly['ΔAIC'].values
akaike_weights = np.exp(-delta_aic/2) / np.sum(np.exp(-delta_aic/2))
df_poly['Akaike Weight'] = akaike_weights

print("\nAkaike Weights (Model Selection Probabilities):")
for i, row in df_poly.iterrows():
    print(f"{row['Model']}: {row['Akaike Weight']:.3f} ({row['Akaike Weight']*100:.1f}%)")



In [ ]:
# Find best models
best_aic_poly = df_poly.loc[df_poly['AIC'].idxmin(), 'Model']
best_bic_poly = df_poly.loc[df_poly['BIC'].idxmin(), 'Model']

print(f"• Best model by AIC: {best_aic_poly}")
print(f"• Best model by BIC: {best_bic_poly}")

if best_aic_poly == best_bic_poly:
    print(f"• Both criteria agree on {best_aic_poly}")
else:
    print("• AIC and BIC disagree - suggests trade-off between fit and complexity")

# Check model uncertainty
max_weight_poly = df_poly['Akaike Weight'].max()
if max_weight_poly > 0.9:
    print(f"• High confidence in model selection (weight = {max_weight_poly:.3f})")
elif max_weight_poly > 0.7:
    print(f"• Moderate confidence in model selection (weight = {max_weight_poly:.3f})")
else:
    print(f"• Substantial uncertainty in model selection (weight = {max_weight_poly:.3f})")


In [ ]:
def fit_multiple_regression(x1, x2, y):
    """Fit multiple regression model: y ~ x1 + x2"""
    X = np.column_stack([np.ones(len(x1)), x1, x2])  # add intercept
    coefficients = np.linalg.lstsq(X, y, rcond=None)[0]
    y_pred = X @ coefficients
    k = len(coefficients)
    
    return y_pred, coefficients, k, X

In [ ]:
# Variable Selection
models_var = {}
results_var = []

# Model 1: Height ~ Age (Linear)
y_pred1, coeffs1, k1, X1 = fit_polynomial_model(data_x, data_y, 1)
aic1, bic1, log_lik1, rss1, sigma2_1 = calculate_aic_bic(data_y, y_pred1, k1, n)

models_var['Height ~ Age'] = {
    'y_pred': y_pred1,
    'k': k1,
    'aic': aic1,
    'bic': bic1,
    'coefficients': coeffs1
}

results_var.append({
    'Model': 'Height ~ Age',
    'Variables': 'Age',
    'Parameters (k)': k1,
    'Log-Likelihood': log_lik1,
    'R²': 1 - rss1 / np.sum((data_y - np.mean(data_y))**2),
    'AIC': aic1,
    'BIC': bic1
})

# Model 2: Height ~ Weight (Linear)
y_pred2, coeffs2, k2, X2 = fit_polynomial_model(data_z, data_y, 1)
aic2, bic2, log_lik2, rss2, sigma2_2 = calculate_aic_bic(data_y, y_pred2, k2, n)

models_var['Height ~ Weight'] = {
    'y_pred': y_pred2,
    'k': k2,
    'aic': aic2,
    'bic': bic2,
    'coefficients': coeffs2
}

results_var.append({
    'Model': 'Height ~ Weight',
    'Variables': 'Weight',
    'Parameters (k)': k2,
    'Log-Likelihood': log_lik2,
    'R²': 1 - rss2 / np.sum((data_y - np.mean(data_y))**2),
    'AIC': aic2,
    'BIC': bic2
})

# Model 3: Height ~ Age + Weight (Multiple regression)
y_pred3, coeffs3, k3, X3 = fit_multiple_regression(data_x, data_z, data_y)
aic3, bic3, log_lik3, rss3, sigma2_3 = calculate_aic_bic(data_y, y_pred3, k3, n)

models_var['Height ~ Age + Weight'] = {
    'y_pred': y_pred3,
    'k': k3,
    'aic': aic3,
    'bic': bic3,
    'coefficients': coeffs3
}

results_var.append({
    'Model': 'Height ~ Age + Weight',
    'Variables': 'Age + Weight',
    'Parameters (k)': k3,
    'Log-Likelihood': log_lik3,
    'R²': 1 - rss3 / np.sum((data_y - np.mean(data_y))**2),
    'AIC': aic3,
    'BIC': bic3
})

# Create results DataFrame for variable selection
df_var = pd.DataFrame(results_var)
print("\nVariable Selection Results:")
print(df_var.round(3))

# Calculate differences
df_var['ΔAIC'] = df_var['AIC'] - df_var['AIC'].min()
df_var['ΔBIC'] = df_var['BIC'] - df_var['BIC'].min()

print("\nModel Comparison (Variable Selection):")
print(df_var[['Model', 'Variables', 'Parameters (k)', 'AIC', 'ΔAIC', 'BIC', 'ΔBIC']].round(3))

# Calculate Akaike weights for variable selection
delta_aic_var = df_var['ΔAIC'].values
akaike_weights_var = np.exp(-delta_aic_var/2) / np.sum(np.exp(-delta_aic_var/2))
df_var['Akaike Weight'] = akaike_weights_var

print("\nAkaike Weights (Variable Selection Probabilities):")
for i, row in df_var.iterrows():
    print(f"{row['Model']}: {row['Akaike Weight']:.3f} ({row['Akaike Weight']*100:.1f}%)")


In [ ]:
# Find best models
best_aic_var = df_var.loc[df_var['AIC'].idxmin(), 'Model']
best_bic_var = df_var.loc[df_var['BIC'].idxmin(), 'Model']

print(f"• Best model by AIC: {best_aic_var}")
print(f"• Best model by BIC: {best_bic_var}")

In [ ]:
# Analyze variable importance
if 'Age + Weight' in best_aic_var:
    print("• Both age and weight appear useful for predicting height")
else:
    print(f"• Single variable model ({best_aic_var.split('~')[1].strip()}) is preferred")

# Check if adding variables is worthwhile
simple_model_idx = df_var[df_var['Variables'] == 'Age'].index[0]
complex_model_idx = df_var[df_var['Variables'] == 'Age + Weight'].index[0]

delta_aic_addition = df_var.loc[complex_model_idx, 'AIC'] - df_var.loc[simple_model_idx, 'AIC']
delta_bic_addition = df_var.loc[complex_model_idx, 'BIC'] - df_var.loc[simple_model_idx, 'BIC']

print(f"• Adding weight to age model changes AIC by {delta_aic_addition:.2f}")
print(f"• Adding weight to age model changes BIC by {delta_bic_addition:.2f}")

if delta_aic_addition < -2:
    print("• Strong evidence that weight improves the model (ΔAIC < -2)")
elif delta_aic_addition > 2:
    print("• Strong evidence against including weight (ΔAIC > 2)")
else:
    print("• Weak evidence about including weight (-2 ≤ ΔAIC ≤ 2)")

In [ ]:
# visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Data and polynomial fits
ax1 = axes[0, 0]
ax1.scatter(data_x, data_y, color='black', s=50, alpha=0.7, label='Data')

# Plot polynomial fits
x_smooth = np.linspace(data_x.min(), data_x.max(), 100)
colors = ['blue', 'red', 'green']
for i, (model_name, model_info) in enumerate(models_poly.items()):
    degree = model_info['degree']
    coeffs = model_info['coefficients']
    
    # Create polynomial features for smooth line
    poly_features = PolynomialFeatures(degree=degree, include_bias=True)
    x_smooth_poly = poly_features.fit_transform(x_smooth.reshape(-1, 1))
    y_smooth = x_smooth_poly @ coeffs
    
    ax1.plot(x_smooth, y_smooth, color=colors[i], 
             label=f'Degree {degree} (AIC={model_info["aic"]:.1f})')

ax1.set_xlabel('Age')
ax1.set_ylabel('Height')
ax1.set_title('Polynomial Model Fits')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: AIC/BIC comparison for polynomial models
ax2 = axes[0, 1]
x_pos = np.arange(len(df_poly))
width = 0.35

bars1 = ax2.bar(x_pos - width/2, df_poly['AIC'], width, label='AIC', alpha=0.8)
bars2 = ax2.bar(x_pos + width/2, df_poly['BIC'], width, label='BIC', alpha=0.8)

ax2.set_xlabel('Polynomial Degree')
ax2.set_ylabel('Information Criterion Value')
ax2.set_title('AIC vs BIC: Polynomial Models')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f'Degree {d}' for d in df_poly['Degree']])
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom')

# Plot 3: Variable selection results
ax3 = axes[1, 0]
x_pos_var = np.arange(len(df_var))
width = 0.35

bars3 = ax3.bar(x_pos_var - width/2, df_var['AIC'], width, label='AIC', alpha=0.8)
bars4 = ax3.bar(x_pos_var + width/2, df_var['BIC'], width, label='BIC', alpha=0.8)

ax3.set_xlabel('Model')
ax3.set_ylabel('Information Criterion Value')
ax3.set_title('AIC vs BIC: Variable Selection')
ax3.set_xticks(x_pos_var)
ax3.set_xticklabels(['Age', 'Weight', 'Age+Weight'], rotation=45)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Add value labels
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom')
for bar in bars4:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom')

# Plot 4: Akaike weights
ax4 = axes[1, 1]

# Combine polynomial and variable selection weights
all_models = list(df_poly['Model']) + list(df_var['Model'])
all_weights = list(df_poly['Akaike Weight']) + list(df_var['Akaike Weight'])

# Separate into two groups for better visualization
poly_models = df_poly['Model'].tolist()
poly_weights = df_poly['Akaike Weight'].tolist()

var_models = [m.split('~')[1].strip() for m in df_var['Model']]
var_weights = df_var['Akaike Weight'].tolist()

x_pos1 = np.arange(len(poly_models))
x_pos2 = np.arange(len(var_models)) + len(poly_models) + 0.5

bars5 = ax4.bar(x_pos1, poly_weights, label='Polynomial Models', alpha=0.8)
bars6 = ax4.bar(x_pos2, var_weights, label='Variable Selection', alpha=0.8)

ax4.set_ylabel('Akaike Weight')
ax4.set_title('Model Selection Probabilities')
ax4.set_xticks(list(x_pos1) + list(x_pos2))
ax4.set_xticklabels([f'Deg {i+1}' for i in range(len(poly_models))] + var_models, rotation=45)
ax4.legend()
ax4.grid(True, alpha=0.3)

# Add percentage labels
for bar in bars5:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height*100:.1f}%', ha='center', va='bottom')
for bar in bars6:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height*100:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# summary
print(f"\nDATASET SUMMARY:")
print(f"• Sample size: n = {n} (small sample)")
print(f"• Variables: Age (predictor), Height (response), Weight (additional predictor)")
print(f"• Age-Height correlation: r = {np.corrcoef(data_x, data_y)[0,1]:.3f}")
print(f"• Weight-Height correlation: r = {np.corrcoef(data_z, data_y)[0,1]:.3f}")
print(f"• Age-Weight correlation: r = {np.corrcoef(data_x, data_z)[0,1]:.3f}")

print(f"\nKEY FINDINGS:")
print(f"1. Polynomial Model Selection:")
print(f"   • AIC prefers: {best_aic_poly}")
print(f"   • BIC prefers: {best_bic_poly}")
print(f"   • Model uncertainty: {'Low' if max_weight_poly > 0.9 else 'Moderate' if max_weight_poly > 0.7 else 'High'}")

print(f"\n2. Variable Selection:")
print(f"   • AIC prefers: {best_aic_var}")
print(f"   • BIC prefers: {best_bic_var}")
print(f"   • Both age and weight are {('strong' if min(delta_aic_addition, delta_bic_addition) < -2 else 'weak')} predictors")

print(f"\n3. Statistical Insights:")
print(f"   • Small sample size (n={n}) means BIC penalty is modest: ln({n}) = {np.log(n):.2f}")
print(f"   • Limited power to distinguish between models")
print(f"   • Results should be interpreted cautiously due to small n")




**Recommendations:**
- For prediction: Use model selected by AIC
- For interpretation: Consider BIC's more parsimonious choice
- Collect more data to reduce model selection uncertainty
- Validate results with cross-validation or independent data

## Back to the Opening Problem

Will be seen as a lab.

## Useful Links
- [Linear Regression by StatQuest](https://www.youtube.com/watch?v=7ArmBVF2dCs)
- [R-squared, Clearly Explained!!! by StatQuest](https://www.youtube.com/watch?v=2AQKmw14mHM)
- [The Main Ideas of Fitting a Line to Data by StatQuest](https://www.youtube.com/watch?v=PaFPbb66DxQ)
- [Regularization Part 1: Ridge (L2) Regression by StatQuest](https://www.youtube.com/watch?v=Q81RR3yKn30)
- [Regularization Part 2: Lasso (L1) Regression by StatQuest](https://www.youtube.com/watch?v=NGf0voTMlcs&t=290s)
- [Regularization Part 3: Elastic Net Regression by StatQuest](https://www.youtube.com/watch?v=1dKRdX9bfIo&t=132s)
- [Ridge vs Lasso Regression, Visualized!!!](https://www.youtube.com/watch?v=Xm2C_gTAl8c)

- [Bias and Variance by StatQuest](https://www.youtube.com/watch?v=EuBBz3bI-aA)
    